In [ ]:
!pip install transformers accelerate huggingface_hub
!pip install -U bitsandbytes

In [ ]:
import json
import csv
import torch
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from torch import cuda, bfloat16
import pandas as pd
import google.generativeai as genai
import os
import time
#from groq import Groq
#import openai
from google.api_core.exceptions import InternalServerError
import re
from huggingface_hub import login
from transformers import (AutoModelForCausalLM, AutoModel,
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
login("")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
#from peft import LoraConfig, PeftConfig
# from trl import SFTTrainer
# from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, AutoModel,
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

from scipy import stats
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve
from sklearn.isotonic import IsotonicRegression       #For calibrating the probs
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import metrics   #Additional scklearn functions

from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn.metrics import RocCurveDisplay



# There are important!
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [5]:
import json
import numpy as np
from scipy import spatial

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,  #False
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16" #"float16", torch.float16,
    #load_in_8bit_fp32_cpu_offload=True,
)

In [ ]:
import torch

# Initialize an empty list to store the embeddings
ArticleEmbedding = []

def compute_article_embedding(embed, tokenizer):
    # Load the dataset from the JSON file
    with open("/kaggle/input/dataset-multi-class-classification/NewsBlog_ProthomAlo_FinalDataset.json", "r") as f:
        data = json.loads(f.read())
        
        # Set the model to evaluation mode (disables dropout, etc.)
        embed.eval()
        
        # Loop through each article in the dataset
        for l, article_data in enumerate(data):
            article = dict()
            print(f"Processing article number: {l + 1}")
            
            # Get the article text
            article_text = article_data['Article Text']
            
            # Tokenize the article text
            inputs = tokenizer(article_text, return_tensors="pt", truncation=True, max_length=256)  # Limit to 512 tokens

            
            # Perform inference without storing gradients
            with torch.no_grad():
                embedding = embed(**inputs, output_hidden_states=True)
                hidden_state = embedding.hidden_states[-1]
            # Take the mean of the embedding along the sequence dimension (dim=1) to get a single vector for the article
            article_embedding = hidden_state.mean(dim=1).squeeze()
            
            # Add the embedding and other details to the article dictionary
            article['Article Text'] = article_text
            article['Article Embedding'] = article_embedding
            article['Label'] = article_data.get('Concept')
            
            # Append the processed article to the list of embeddings
            ArticleEmbedding.append(article)
            torch.cuda.empty_cache()
    return ArticleEmbedding


In [10]:
def compute_label_embedding(embed, tokenizer):
    print("inside label embedding")
    with open("/kaggle/input/dataset-multi-class-classification/Keyword_BengaliNews.json", "r") as f:
        key = json.loads(f.read())
        
        for i in range(len(key)):
            label_embedding = 0
            label = key[i]['Keyword'][0]
            
            for j in range(len(key[i]['Keyword'])):
                # Tokenize the keyword
                inputs = tokenizer(key[i]['Keyword'][j], return_tensors="pt", truncation=True, max_length=512)
                
                # Perform inference without gradients
                with torch.no_grad():
                    # Forward pass through the model
                    outputs = embed(**inputs, output_hidden_states=True)
                    
                    # Extract the last hidden state tensor
                    hidden_state = outputs.hidden_states[-1]  # This gives you the tensor
                    
                    # Mean pooling across the sequence dimension (dim=1)
                    keyword_embedding = hidden_state.mean(dim=1).squeeze()
                    
                    # Accumulate embeddings
                    label_embedding += keyword_embedding
            
            # Store averaged embeddings
            LabelEmbedding[label] = label_embedding / len(key[i]['Keyword'])
    
    return LabelEmbedding


In [ ]:
dataset_path="/kaggle/input/dataset-multi-class-classification/NewsBlog_ProthomAlo_FinalDataset.json"
result_path="/kaggle/working/output(1000-2000)"

#torch.cuda.empty_cache()
#device_map={"": 0}
#Load the model here
base_model = "BanglaLLM/bangla-llama-13b-instruct-v0.1" 

base_model_name = base_model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)
model.gradient_checkpointing_enable()
torch.cuda.empty_cache()

model.config.use_cache = False
model.config.pretraining_tp = 1
# model = AutoModel.from_pretrained(base_model_name, device_map="auto", offload_folder="offload")
# model.gradient_checkpointing_enable()
# Load the tokenizer here if needed
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True, trust_remote_code=True)

#LaBSE implementation
# model = SentenceTransformer('sentence-transformers/LaBSE')

#LASER implementation
# laser = Laser()

# Bangla transformer implementation
# transformer=Bangla_sentence_transformer_small()


#Bloom implementation
# tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b1")
# model = BloomModel.from_pretrained("bigscience/bloomz-1b1")


#Flan-UL2 implementation
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-ul2")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

#GPTNeoX implementation
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
# model = GPTNeoXModel.from_pretrained("EleutherAI/gpt-neox-20b")
LabelEmbedding={}
ArticleEmbedding=[]

LabelEmbedding= compute_label_embedding2(model, tokenizer)
ArticleEmbedding= compute_article_embedding(model, tokenizer)



In [ ]:


for threshold in np.arange(0.95,0.99,0.05):
        final_article_list = []
        print("Threshold running for::",threshold)
        filename = str(threshold) + '_NewsBlog_ProthomAlo_Dataset.json'
        for i in range(0, len(ArticleEmbedding)):
            inferred_label=[]
            final_article = {}
            for label in LabelEmbedding:
                x = ArticleEmbedding[i]['Article Embedding']
                #.hidden_states[-1]
                label_embedding_np = LabelEmbedding[label].cpu().numpy().flatten()
                squeezed_embedding_np = x.flatten()
                squeezed_embedding_np = [np.mean(squeezed_embedding_np)]
                label_embedding_np = [np.mean(label_embedding_np)]
                # print(label_embedding_np)
                # print(squeezed_embedding_np)
                cosine_dist = 1 - spatial.distance.cosine(label_embedding_np, squeezed_embedding_np)
                #print(cosine_dist)
                if (cosine_dist > threshold):
                    inferred_label.append(label)
        
            final_article['Article Text'] = ArticleEmbedding[i]['Article Text']
            final_article['Inferred Article Label'] = inferred_label # inferred labels
            final_article['Label'] = ArticleEmbedding[i]['Label']  # ground truth labels added for evaluation
            final_article_list.append(final_article)

        with open(result_path+filename, 'w', encoding='utf-8', newline='\n') as f:
            #print("writing json::",filename)
            json.dump(final_article_list, f, ensure_ascii=False, indent=4, separators=(',', ':'))
            